此 ipynb 文件只用于内部演示，往 GitHub 提 PR 前会删除。

In [1]:
model_name = 'ced-mini'  # ['ced-tiny', 'ced-mini', 'ced-small', 'ced-base']
model_saved_path = f'/home/zhangjunbo/sandbox/{model_name}'

In [2]:
# Convert checkpoint from original CED to 🤗 Transformers format
from transformers.models.ced.convert_ced_original_to_pytorch import convert_ced_checkpoint

convert_ced_checkpoint(model_name, model_saved_path)

Saving model ced-mini to /home/zhangjunbo/sandbox/ced-mini
No model name specified for CedConfig, use default settings.
Configuration saved in /home/zhangjunbo/sandbox/ced-mini/config.json
Model weights saved in /home/zhangjunbo/sandbox/ced-mini/pytorch_model.bin
Feature extractor saved in /home/zhangjunbo/sandbox/ced-mini/preprocessor_config.json


In [3]:
# TODO: support AutoModel and AutoFeatureExtractor

# from transformers import AutoModelForAudioFrameClassification, AutoFeatureExtractor
# model = AutoModelForAudioFrameClassification.from_pretrained(model_saved_path)
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_saved_path)

In [ ]:
from transformers import CedConfig, CedForAudioClassification, CedFeatureExtractor

front_end = CedFeatureExtractor()
model = CedForAudioClassification(CedConfig(model_name)).from_pretrained(model_saved_path).eval()

In [5]:
import torchaudio

wavpath = '/home/zhangjunbo/src/CED/resources/JeD5V5aaaoI_931_932.wav'
wave, _ = torchaudio.load(wavpath)

In [6]:
feature = front_end(wave)["input_values"]
output = model(feature).squeeze(0)

In [7]:
import pandas as pd

label_maps = pd.read_csv('http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv').set_index('index')['display_name'].to_dict()

topk = 3

for k, (prob, label) in enumerate(zip(*output.topk(topk))):
    lab_idx = label.item()
    label_name = label_maps[lab_idx]
    print(f"Topk-{k+1} {label_name:<30} {prob:.4f}")

Topk-1 Finger snapping                0.9068
Topk-2 Slap, smack                    0.0610
Topk-3 Inside, small room             0.0426
